In [1]:
# Baixando
!pip install -U -q PyDrive
!pip install dataprep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 4.9 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 1.0 MB 40.9 MB/s 
     |████████████████████████████████| 133 kB 36.1 MB/s 
     |████████████████████████████████| 965 kB 42.6 MB/s 
     |████████████████████████████████| 28.5 MB 2.8 MB/s 
     |████████████████████████████████| 943 kB 31.1 MB/s 
     |████████████████████████████████| 78 kB 3.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 749 kB 54.2 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 232 kB 47.0 MB/s 
     |████████████████████████████████| 1.6 MB 61.1 MB/s 
     |██████████████████████

In [2]:
# Importando bibliotecas
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from sklearn import datasets
from dataprep.eda import create_report
from oauth2client.client import GoogleCredentials
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.preprocessing import LabelEncoder #Scikit Learn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [3]:
# Autenticando para entrar no drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Baixando o arquivo de metadados
downloaded = drive.CreateFile({'id': '1AUBksb9ooahhbiqi1h0DoAhyPxpylpvo'})
downloaded.GetContentFile('book_variaveis_pan_inteli_v4.csv')
print(downloaded['title'], downloaded['mimeType'])

In [ ]:
# Lendo os dados do arquivo
data = pd.read_csv('book_variaveis_pan_inteli_v4.csv')

In [ ]:
# Seleção da safra do ano de 2021 no mês 11
data_with_safra = data.loc[(data.anomes == 202111)]

In [ ]:
# Tira as linhas duplicadas da safra
data_with_safra.drop_duplicates()

In [ ]:
# retira as linhas sem saldo ou cod_rating
data_without_clients = data_with_safra.dropna(subset=['vlr_saldo','cod_rating'])

In [ ]:
# seleciona apenas as linhas com atrito 1
data_with_friction = data_without_clients.loc[(data_without_clients['ind_atrito'] == 1)]

In [ ]:
# seleciona as colunas necessárias para o modelo
cat_cols = ['anomes', 'vlr_score', 'vlr_saldo', 'num_atend', 'num_atend_atrs', 'qtd_oper', 'qtd_reclm', 'cod_rating' , 'ind_atrito', 'qtd_restr' ]
data_with_cat_cols = data_with_friction[cat_cols]

In [ ]:
# Substitui os valores NaN pelo valor 0
data_without_nan = data_with_cat_cols.fillna(0)

In [ ]:
# criacao de uma variável que armazena os métodos do label encoder
le = LabelEncoder()
# assinalando valores númericos para uma nova coluna chamada cod_rating_numeric, assim, mapeando os dados categóricos
data_without_nan['cod_rating_numeric'] = le.fit_transform(data_without_nan['cod_rating'])
# a variável 'A' virou 0

In [ ]:
# dropando colunas que auxiliaram no tratamento, mas que não vão ser usadas no dataf
data_without_nan = data_without_nan.drop(['anomes','cod_rating'], axis = 1)

In [ ]:
x = data_without_nan 

In [ ]:
# CONJUNTO DE DADOS NÃO ATRITADOS
# Seleciona linhas com clientes sem atrito para o novo conjunto
data_not_friction = data_without_clients.loc[(data_without_clients['ind_atrito'].isnull())]
data_not_friction

In [ ]:
# Seleciona as colunas necessárias no conjunto novo e tira os valores NaN
data_not_friction_columns = data_not_friction[cat_cols].fillna(0)

In [ ]:
# Transforma o dado categórico da coluna cod_rating para numérico do novo conjunto de dados
data_not_friction_columns['cod_rating_numeric'] = le.fit_transform(data_not_friction_columns['cod_rating'])

In [ ]:
# Retira as colunas que só auxiliaram na preparação dos dados  
data_not_friction_columns = data_not_friction_columns.drop(['anomes','cod_rating'], axis = 1)

In [ ]:
# O conjunto y representa o conjunto não atritado
y = data_not_friction_columns.head(438)

In [ ]:
# Junção de dois conjunto de dados
pre_frame = x.append(y)

In [ ]:
# Agrupamento, de forma que sempre varie a ordem das linhas
safra_artificial = pre_frame.sample(frac=1)
safra_artificial

In [ ]:
create_report(safra_artificial).show()

In [ ]:
safra_artificial.to_csv('safra_artificial.csv')

In [ ]:
from google.colab import files
files.download("safra_artificial.csv")

###Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
X, y = np.arange(10).reshape((5, 2)), range(5)
X

In [ ]:
list(y)

In [ ]:
data = safra_artificial

###Iris

In [ ]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
iris

### Criando árvore de decisão

> Bloco com recuo



In [ ]:
x = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                    random_state=42)

model = DecisionTreeClassifier(criterion='entropy', random_state=42)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)
y_pred

### Avaliando os testes

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
def accuracy(yt, yp):
    return (yp==yt).mean()

accuracy(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
_ = plot_confusion_matrix(model, x_test, y_pred, cmap='Blues')#, normalize = 'true')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Dividindo x e y
x = data.drop(columns="ind_atrito")
y = data['ind_atrito']

# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

# Treinando o modelo
model = LogisticRegression().fit(x_train, y_train)

# Fazendo as predições
y_pred = model.predict(x_test)

###Matriz de confusão


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
_ = plot_confusion_matrix(model, x_test, y_pred, cmap='Blues')#, normalize = 'true')